In [1]:
!pip install visualkeras gradio tensorflow

In [2]:
import sys
assert sys.version_info >= (3, 8)

import numpy as np
import gradio as gr
import cv2 as cv
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess

if cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

In [3]:
vgg16_model = tf.keras.models.load_model("G:/My Drive/Colab Notebooks/model/VGG-16_Model.keras")
vgg16_model_fine_tuned = tf.keras.models.load_model("G:/My Drive/Colab Notebooks/model/VGG-16_Model_Fine_Tuned.keras")

resnet50_model = tf.keras.models.load_model("G:/My Drive/Colab Notebooks/model/ResNet50_Model.keras")
resnet50_model_fine_tuned = tf.keras.models.load_model("G:/My Drive/Colab Notebooks/model/ResNet50_Model_Fine_Tuned.keras")

mobilenetv2_model = tf.keras.models.load_model("G:/My Drive/Colab Notebooks/model/MobileNetV2_Model.keras")
mobilenetv2_model_fine_tuned = tf.keras.models.load_model("G:/My Drive/Colab Notebooks/model/MobileNetV2_Model_Fine_Tuned.keras")


In [4]:
category = {"000" : "Speed limit 5km/h",
            "001" : "Speed limit 15km/h",
            "002" : "Speed limit 30km/h",
            "003" : "Speed limit 40km/h",
            "004" : "Speed limit 50km/h",
            "005" : "Speed limit 60km/h",
            "006" : "Speed limit 70km/h",
            "007" : "Speed limit 80km/h",
            "008" : "No turn left or go straight",
            "009" : "No turn right or go straight",
            "010" : "No go straight",
            "011" : "No turn left",
            "012" : "No turn left or right",
            "013" : "No turn right",
            "014" : "No overtaking",
            "015" : "No u-turn",
            "016" : "No entry for vehicles",
            "017" : "No horn",
            "018" : "Speed limit 40KM/H ended",
            "019" : "Speed limit 50KM/H ended",
            "020" : "Turn right or go straight",
            "021" : "Go straight",
            "022" : "Turn left",
            "023" : "Turn left or turn right",
            "024" : "Turn Right",
            "025" : "Stay at the left lane",
            "026" : "Stay at the right lane",
            "027" : "Roundabout",
            "028" : "Car lane",
            "029" : "Horn allowed",
            "030" : "Bicycle lane",
            "031" : "U-turn allowed",
            "032" : "Bypass left and right",
            "033" : "Traffic light ahead",
            "034" : "Beware of danger",
            "035" : "Beware of pedestrian",
            "036" : "Beware of bicycles",
            "037" : "Beware of students",
            "038" : "Sharp right turn",
            "039" : "Sharp left turn",
            "040" : "Downhill road",
            "041" : "Uphill road",
            "042" : "Slow down",
            "043" : "Right lane T-junction",
            "044" : "Left lane T-junction",
            "045" : "Village or Residential Area Ahead",
            "046" : "Winding Road",
            "047" : "Unguarded railway crossings",
            "048" : "Construction ahead",
            "049" : "Continuous turn road",
            "050" : "Guarded railway crossing ahead",
            "051" : "Accident prone ahead",
            "052" : "Stop sign",
            "053" : "No vehicle",
            "054" : "Parking prohibited",
            "055" : "No entry",
            "056" : "Give way",
            "057" : "Stop for roadblock",
           }

In [ ]:
def preprocess(image, model_name):
    image = cv.resize(image, (64, 64))  # Resize image to 64x64
    image = image.astype('float32')  # Convert image to float

    if model_name in ['VGG16', 'VGG16-Fine-Tuned']:
        image = vgg16_preprocess(image)  # VGG16 specific preprocessing
    elif model_name in ['ResNet50', 'ResNet50-Fine-Tuned']:
        image = resnet_preprocess(image)  # ResNet50 specific preprocessing
    elif model_name in ['MobileNetV2', 'MobileNetV2-Fine-Tuned']:
        image = mobilenet_preprocess(image)  # MobileNetV2 specific preprocessing

    image = np.expand_dims(image, axis=0) 
    return image

def predict(model_name, image):
    if model_name == 'VGG16':
        model = vgg16_model
    elif model_name == 'VGG16-Fine-Tuned':
        model = vgg16_model_fine_tuned
    elif model_name == 'ResNet50':
        model = resnet50_model
    elif model_name == 'ResNet50-Fine-Tuned':
        model = resnet50_model_fine_tuned
    elif model_name == 'MobileNetV2':
        model = mobilenetv2_model
    elif model_name == 'MobileNetV2-Fine-Tuned':
        model = mobilenetv2_model_fine_tuned

    # Preprocess the image
    image = preprocess(image, model_name)

    # Get the prediction and probability
    outputs = model.predict(image)
    predicted_class = np.argmax(outputs, axis=1)[0]
    probability = np.max(outputs, axis=1)[0]  # Get the probability of the predicted class

    # Convert the predicted class to a string key for the category dictionary
    predicted_class_str = f"{predicted_class:03}"

    # Get the label from the category dictionary
    predicted_label = category.get(predicted_class_str, "Unknown Sign")

    # Prepare the label text with the predicted label and probability
    label_text = f'Predicted Class: {predicted_class_str} - {predicted_label}\nProbability: {probability:.2f}'

    return label_text

interface = gr.Interface(
    fn=predict,
    inputs=[
        gr.Dropdown(['VGG16', 'VGG16-Fine-Tuned', 'ResNet50', 'ResNet50-Fine-Tuned', 'MobileNetV2', 'MobileNetV2-Fine-Tuned'], label="Choose Model"),
        gr.Image(type="numpy", label="Upload Traffic Sign Image")
    ],
    outputs=[
        gr.Textbox(label="Prediction"),
    ],
    title="Traffic Sign Classifier",
    description="Choose a model (VGG16, ResNet50, or MobileNetV2), upload a traffic sign image, and let the model classify it."
)

interface.launch(debug=True)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
